<a href="https://colab.research.google.com/github/HardworkingPearl/VCC-state/blob/evo2emb/Experiments/evo2embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install anndata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.3/169.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 6.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive/VCC/datasets/competition_support_set /content/drive/MyDrive/VCC/datasets/vcc_data

/content/drive/MyDrive/VCC/datasets/competition_support_set:
competition_train.h5	       gene_names.csv  k562_gwps.h5  starter.toml
competition_val_template.h5ad  hepg2.h5        k562.h5
ESM2_pert_features.pt	       jurkat.h5       rpe1.h5

/content/drive/MyDrive/VCC/datasets/vcc_data:
adata_Training.h5ad  pert_counts_Validation.csv
gene_names.csv	     TEST_small.h5ad


In [5]:
competition_support_set_path = "/content/drive/MyDrive/VCC/datasets/competition_support_set"
vcc_data_path = "/content/drive/MyDrive/VCC/datasets/vcc_data"

In [6]:
import h5py
import numpy as np
filename = f"{competition_support_set_path}/k562.h5"

with h5py.File(filename, "r") as f:
  for key in f.keys():
    print(key)
    print(np.array(f[key]))

X
[[0.        1.7592828 0.        ... 3.7345767 2.7359772 5.5159492]
 [0.        0.        0.        ... 3.57988   2.033628  5.394437 ]
 [0.        1.3249131 0.        ... 3.3046005 2.7844076 4.9049053]
 ...
 [0.        0.        0.        ... 3.6838982 3.1109695 4.9110603]
 [0.        0.8855004 0.        ... 3.6001828 2.2560368 5.0871906]
 [0.        1.5953873 0.        ... 3.0742545 2.1816006 5.0639205]]
layers
[]
obs
['UMI_count' 'batch_var' 'cell_barcode' 'cell_type'
 'core_adjusted_UMI_count' 'core_scale_factor' 'gem_group' 'gene'
 'gene_id' 'gene_transcript' 'mitopercent' 'sgID_AB' 'target_gene'
 'transcript' 'z_gemgroup_UMI']
obsm
[]
obsp
[]
uns
[]
var
['_index']
varm
[]
varp
[]


In [76]:
import anndata
adata = anndata.read_h5ad(f"{vcc_data_path}/TEST_small.h5ad")
adata

AnnData object with n_obs × n_vars = 3 × 18080
    obs: 'target_gene', 'guide_id', 'batch'
    var: 'gene_id'

In [77]:
adata.obs

,target_gene,guide_id,batch
AAACAAGCAACCTTGTACTTTAGG-Flex_1_01,CHMP3,CHMP3_P1P2_A|CHMP3_P1P2_B,Flex_1_01
AAACAAGCATTGCCGCACTTTAGG-Flex_1_01,AKT2,AKT2_P1P2_A|AKT2_P1P2_B,Flex_1_01
AAACCAATCAATGTTCACTTTAGG-Flex_1_01,SHPRH,SHPRH_P1P2_A|SHPRH_P1P2_B,Flex_1_01


In [78]:
adata.var

,gene_id
SAMD11,ENSG00000187634
NOC2L,ENSG00000188976
KLHL17,ENSG00000187961
PLEKHN1,ENSG00000187583
PERM1,ENSG00000187642
...,...
MT-ND4L,ENSG00000212907
MT-ND4,ENSG00000198886
MT-ND5,ENSG00000198786
MT-ND6,ENSG00000198695


In [79]:
import pandas as pd
import scipy.sparse as sp

In [80]:
adata.obs["is_control"] = adata.obs["target_gene"].eq("non-targeting")
adata.obs["cell_barcode"] = adata.obs_names.str.split("-").str[0]

# Use integer to encode

## Target Gene

In [81]:
genes = adata.var_names.to_series()
gene_to_col = pd.Series(index=genes.values, data=np.arange(genes.size))
gene_to_col

,0
SAMD11,0
NOC2L,1
KLHL17,2
PLEKHN1,3
PERM1,4
...,...
MT-ND4L,18075
MT-ND4,18076
MT-ND5,18077
MT-ND6,18078


In [82]:
tg = adata.obs["target_gene"].astype(str)
col_idx = tg.map(gene_to_col).fillna(-1).astype(int).to_numpy()
col_idx

array([ 2305, 15684,  6365])

In [83]:
n_cells, n_genes = adata.n_obs, adata.n_vars
rows = np.flatnonzero(col_idx >= 0)
cols = col_idx[col_idx >= 0]
data = np.ones_like(rows, dtype=np.float32)
gene_onehot = sp.csr_matrix((data, (rows, cols)), shape=(n_cells, n_genes), dtype=np.float32)
gene_onehot

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 3 stored elements and shape (3, 18080)>

In [84]:
adata.obsm["onehot_target_gene"] = gene_onehot
adata.uns["onehot_target_gene_spec"] = {
    "columns_align_with": "adata.var_names",
    "control_rule": "obs['target_gene'] == 'non-targeting' -> all zeros",
}

## Batch

In [85]:
adata.obs["batch_cat"] = adata.obs["batch"].astype("category")
batch_categories = adata.obs["batch_cat"].cat.categories.tolist()
batch_code = adata.obs["batch_cat"].cat.codes.to_numpy()
batch_code


array([0, 0, 0], dtype=int8)

In [86]:
rows = np.arange(n_cells, dtype=int)
cols = batch_code
data = np.ones(n_cells, dtype=np.float32)
batch_onehot = sp.csr_matrix((data, (rows, cols)), shape=(n_cells, len(batch_categories)), dtype=np.float32)
batch_onehot

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 3 stored elements and shape (3, 1)>

In [87]:
adata.obsm["onehot_batch"] = batch_onehot
adata.uns["onehot_batch_spec"] = {"columns": batch_categories}

## Guide

In [88]:
guides_series = adata.obs["guide_id"].astype(str).str.split("|")
guides_series

,guide_id
AAACAAGCAACCTTGTACTTTAGG-Flex_1_01,"[CHMP3_P1P2_A, CHMP3_P1P2_B]"
AAACAAGCATTGCCGCACTTTAGG-Flex_1_01,"[AKT2_P1P2_A, AKT2_P1P2_B]"
AAACCAATCAATGTTCACTTTAGG-Flex_1_01,"[SHPRH_P1P2_A, SHPRH_P1P2_B]"


In [89]:
unique_guides = pd.Index(sorted(set(g for lst in guides_series for g in lst)))
unique_guides

Index(['AKT2_P1P2_A', 'AKT2_P1P2_B', 'CHMP3_P1P2_A', 'CHMP3_P1P2_B',
       'SHPRH_P1P2_A', 'SHPRH_P1P2_B'],
      dtype='object')

In [90]:
guide_to_col = pd.Series(index=unique_guides, data=np.arange(len(unique_guides)))
guide_to_col

,0
AKT2_P1P2_A,0
AKT2_P1P2_B,1
CHMP3_P1P2_A,2
CHMP3_P1P2_B,3
SHPRH_P1P2_A,4
SHPRH_P1P2_B,5


In [91]:
row_idx, col_idx, data_vals = [], [], []
for i, lst in enumerate(guides_series):
    for g in lst:
        j = guide_to_col.get(g)
        if pd.notna(j):
            row_idx.append(i)
            col_idx.append(int(j))
            data_vals.append(1.0)
guide_multihot = sp.csr_matrix(
    (np.array(data_vals, dtype=np.float32), (np.array(row_idx), np.array(col_idx))),
    shape=(n_cells, len(unique_guides)),
    dtype=np.float32,
)

In [92]:
guide_multihot.toarray()

array([[0., 0., 1., 1., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1.]], dtype=float32)

In [93]:
adata.obsm["multihot_guide_id"] = guide_multihot
adata.uns["multihot_guide_id_spec"] = {"columns": unique_guides.tolist(), "separator": "|"}

## Combined

In [94]:
adata.obsm["X_meta"] = sp.hstack([gene_onehot, batch_onehot, guide_multihot], format="csr")
adata.uns["encoding_summary"] = {
    "onehot_target_gene_cols": "adata.var_names (length = %d)" % n_genes,
    "onehot_batch_cols": batch_categories,
    "multihot_guide_id_cols": unique_guides.tolist(),
    "control_flag": "obs['is_control']",
}

In [95]:
adata

AnnData object with n_obs × n_vars = 3 × 18080
    obs: 'target_gene', 'guide_id', 'batch', 'is_control', 'cell_barcode', 'batch_cat'
    var: 'gene_id'
    uns: 'onehot_target_gene_spec', 'onehot_batch_spec', 'multihot_guide_id_spec', 'encoding_summary'
    obsm: 'onehot_target_gene', 'onehot_batch', 'multihot_guide_id', 'X_meta'

In [98]:
adata.obs

,target_gene,guide_id,batch,is_control,cell_barcode,batch_cat
AAACAAGCAACCTTGTACTTTAGG-Flex_1_01,CHMP3,CHMP3_P1P2_A|CHMP3_P1P2_B,Flex_1_01,False,AAACAAGCAACCTTGTACTTTAGG,Flex_1_01
AAACAAGCATTGCCGCACTTTAGG-Flex_1_01,AKT2,AKT2_P1P2_A|AKT2_P1P2_B,Flex_1_01,False,AAACAAGCATTGCCGCACTTTAGG,Flex_1_01
AAACCAATCAATGTTCACTTTAGG-Flex_1_01,SHPRH,SHPRH_P1P2_A|SHPRH_P1P2_B,Flex_1_01,False,AAACCAATCAATGTTCACTTTAGG,Flex_1_01


In [99]:
adata.uns

OrderedDict([('onehot_target_gene_spec',
              {'columns_align_with': 'adata.var_names',
               'control_rule': "obs['target_gene'] == 'non-targeting' -> all zeros"}),
             ('onehot_batch_spec', {'columns': ['Flex_1_01']}),
             ('multihot_guide_id_spec',
              {'columns': ['AKT2_P1P2_A',
                'AKT2_P1P2_B',
                'CHMP3_P1P2_A',
                'CHMP3_P1P2_B',
                'SHPRH_P1P2_A',
                'SHPRH_P1P2_B'],
               'separator': '|'}),
             ('encoding_summary',
              {'onehot_target_gene_cols': 'adata.var_names (length = 18080)',
               'onehot_batch_cols': ['Flex_1_01'],
               'multihot_guide_id_cols': ['AKT2_P1P2_A',
                'AKT2_P1P2_B',
                'CHMP3_P1P2_A',
                'CHMP3_P1P2_B',
                'SHPRH_P1P2_A',
                'SHPRH_P1P2_B'],
               'control_flag': "obs['is_control']"})])

In [102]:
adata.obsm['X_meta']

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 12 stored elements and shape (3, 18087)>

#  Directly encode gene name

In [105]:
from transformers import BertTokenizer, BertModel
import torch
BERT_NAME = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(BERT_NAME)
bert = BertModel.from_pretrained(BERT_NAME)

with torch.no_grad():
  input_tokens = tokenizer(["AAACAAGCAACCTTGTACTTTAGG", "AAACAAGCATTGCCGCACTTTAGG", "AAACCAATCAATGTTCACTTTAGG"], add_special_tokens=False, padding=True, return_tensors='pt')
  column_emb = bert(**input_tokens).last_hidden_state
  column_attn_mask = ~input_tokens["attention_mask"].bool()
column_emb

tensor([[[-3.9322e-01, -2.4708e-01,  1.0298e+00,  ...,  1.4616e-02,
          -4.7308e-01,  4.9878e-01],
         [-4.8441e-01, -9.2471e-02,  1.4105e+00,  ..., -4.0588e-02,
          -3.9473e-01,  5.5276e-01],
         [-4.7766e-01, -1.9966e-01,  1.1461e+00,  ..., -1.1774e-01,
          -4.4586e-01,  5.6658e-01],
         ...,
         [-4.0712e-01, -1.9603e-01,  1.2046e+00,  ..., -7.3962e-02,
          -2.9701e-01,  5.9376e-01],
         [-3.3723e-01, -2.4828e-01,  1.2061e+00,  ...,  3.6350e-03,
          -2.8473e-01,  5.5181e-01],
         [-3.9588e-01, -3.7788e-01,  1.2820e+00,  ..., -8.6542e-02,
          -3.6543e-01,  6.3675e-01]],

        [[-4.1928e-01, -2.3813e-01,  9.9893e-01,  ...,  1.3522e-01,
          -5.4043e-01,  6.0587e-01],
         [-5.1767e-01, -1.0881e-01,  1.3665e+00,  ...,  7.3431e-02,
          -4.2581e-01,  5.9541e-01],
         [-3.5055e-01, -1.6263e-01,  1.1945e+00,  ..., -3.7285e-03,
          -4.8961e-01,  6.3646e-01],
         ...,
         [-4.4744e-01, -2

In [106]:
column_attn_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
         False],
        [False, False, False, False, False, False, False, False, False, False,
         False],
        [False, False, False, False, False, False, False, False, False, False,
          True]])

In [108]:
column_emb[0].shape

torch.Size([11, 768])

In [110]:
from transformers import BertTokenizer, BertModel
import torch
BERT_NAME = "zhihan1996/DNA_bert_6"
tokenizer = BertTokenizer.from_pretrained(BERT_NAME)
bert = BertModel.from_pretrained(BERT_NAME)

with torch.no_grad():
  input_tokens = tokenizer(["AAACAAGCAACCTTGTACTTTAGG", "AAACAAGCATTGCCGCACTTTAGG", "AAACCAATCAATGTTCACTTTAGG"], add_special_tokens=False, padding=True, return_tensors='pt')
  column_emb = bert(**input_tokens).last_hidden_state
  column_attn_mask = ~input_tokens["attention_mask"].bool()
column_emb

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

tensor([[[-1.1656, -0.2959, -0.0215,  ..., -1.0763, -1.4064, -0.0761]],

        [[-1.1656, -0.2959, -0.0215,  ..., -1.0763, -1.4064, -0.0761]],

        [[-1.1656, -0.2959, -0.0215,  ..., -1.0763, -1.4064, -0.0761]]])

In [111]:
column_emb[0].shape

torch.Size([1, 768])